In [52]:
# install giphy_client library
import sys
!{sys.executable} -m pip install giphy_client

In [ ]:
!unzip checkpoint.zip

In [ ]:
import csv
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Activation, Flatten, Input, Concatenate
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow_addons as tfa 
import matplotlib.pyplot as plt

In [ ]:
threshold = 0.01
hidden_size = 100
word_embedding_dim = 300
epochs = 10
use_unlabeled_dataset = True

labeled_dataset_size = 1830
train_dataset_size = 900
validation_dataset_size = 100
test_dataset_size = 830
unlabeled_dataset_size = 4000

pos_list = np.char.lower(["ADJ","ADP","ADV","AUX","CONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"])
dep_list = np.char.lower(["ROOT", "acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nmod", "npadvmod", "nsubj", "nsubjpass", "nummod", "oprd", "parataxis", "pcomp", "pobj", "poss", "preconj", "predet", "prep", "prt", "punct", "quantmod", "relcl", "xcomp"])

pos_dim = len(pos_list)
dep_dim = len(dep_list)

filename = "processed-unlabeled-tweet-1.csv"

In [ ]:
# Helper function to one-hot encode the labels
def one_hot(vec, dic):
    vec = np.char.lower(vec)
    return np.array([dic == row for row in vec], dtype='i1')

In [ ]:
with open(filename, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    data = [tuple(x) for x in spamreader]
    data = np.array(data, dtype=([("text", 'U20'),("simplified_text", 'U20'), ("best_match", 'U20'), ("index", int), ("pos", 'U20'), ("dep", 'U20'), ("stop", 'U5')]))

In [ ]:
inputs = Input(shape=(None, pos_dim+dep_dim+1))
x = Embedding(380000, word_embedding_dim)(inputs[:,:,0])
x = Concatenate(axis=-1)([inputs[:,:,1:], x])
x = Bidirectional(LSTM(100, return_sequences=True))(x)
outputs = Dense(2, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

opt = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False
)
model.compile(loss=BinaryCrossentropy(), optimizer=opt)

In [ ]:
model.load_weights('./checkpoint')
prediction = model.predict(data)

In [ ]:
print(prediction)

## Fetch GIFs using GIPHY API

In [51]:
import time
import giphy_client
from giphy_client.rest import ApiException
from pprint import pprint
from IPython.display import HTML

# create an instance of the API class
api_instance = giphy_client.DefaultApi()
api_key = 'dc6zaTOxFJmzC' # str | Giphy API Key.
q = 'cheeseburgers' # str | Search query term or prhase.
limit = 5 # int | The maximum number of records to return. (optional) (default to 25)
offset = 0 # int | An optional results offset. Defaults to 0. (optional) (default to 0)
rating = 'g' # str | Filters results by specified rating. (optional)
lang = 'en' # str | Specify default country for regional content; use a 2-letter ISO 639-1 country code. See list of supported languages <a href = \"../language-support\">here</a>. (optional)
fmt = 'json' # str | Used to indicate the expected response format. Default is Json. (optional) (default to json)

# Search Endpoint
api_response = api_instance.gifs_search_get(api_key, q, limit=limit, offset=offset, rating=rating, lang=lang, fmt=fmt)
urls = []
for i in range(len(api_response.data)):
    urls.append(api_response.data[i].images.downsized.url)
print(urls)

img = ""

for i in range(len(urls)):
    img += "<img src='" + urls[i] + "'>"

HTML(img)

['https://media2.giphy.com/media/KpLPyE3D6HJPa/giphy.gif?cid=e1bb72ff6rz05d67r4bqflr0jju1z44cc1yv9m5mc7hlj6m9&rid=giphy.gif', 'https://media0.giphy.com/media/uQvxobRExS9nG/giphy.gif?cid=e1bb72ff6rz05d67r4bqflr0jju1z44cc1yv9m5mc7hlj6m9&rid=giphy.gif', 'https://media1.giphy.com/media/elxZwVNCcL5hvktNFs/giphy.gif?cid=e1bb72ff6rz05d67r4bqflr0jju1z44cc1yv9m5mc7hlj6m9&rid=giphy.gif', 'https://media2.giphy.com/media/26AHEhWPgcK6xOU7K/giphy.gif?cid=e1bb72ff6rz05d67r4bqflr0jju1z44cc1yv9m5mc7hlj6m9&rid=giphy.gif', 'https://media1.giphy.com/media/sEjcayPGhu1Xi/giphy.gif?cid=e1bb72ff6rz05d67r4bqflr0jju1z44cc1yv9m5mc7hlj6m9&rid=giphy.gif']
